In [7]:
import numpy as np
import pickle

from classes.agent import Agent
from classes.ou_network import OU_Network
from classes.internal_state import Normative_DIS
from classes.action_state import Discounted_gain_soft_horizon_TSAS
from classes.sensory_state import Omniscient_ST

from classes.experiment import Experiment

from methods.policies import softmax_policy_init
from methods.empirical_priors import discrete_empirical_priors


In [8]:
## Import behavioural experiment
with open('/home/vbtesh/Documents/CompProjects/vbtCogSci/csl2analysis/data/csl_2_modelling_data.obj', 'rb') as inFile:
    modelling_data = pickle.load(inFile)


In [9]:
part_key = '5ef109c89196fa6d5cf6c005'
conditions = ['generic', 'congruent', 'incongruent', 'implausible']
cond = conditions[2]


In [10]:
# Model fitting
fitting = True # If false, no data will be used 
if fitting:
    ## Data from trial
    part_data = modelling_data[part_key]['trials'][cond]
    print(part_data.keys())
    
    posterior = part_data['posterior']
    data = part_data['data']
    inters = part_data['inters']
    inters_fit = part_data['inters_fit']
# General model parameters (true for all trials)
if fitting:
    N = data.shape[0] - 1
    K = data.shape[1]
else:
    N = 100
    K = 3
links = np.array([-1, -0.5, 0, 0.5, 1])
theta = 0.5
dt = 0.2
sigma = 1 
# Set up priors
flat_prior = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
flat_prior = np.tile(flat_prior, (6, 1))
random_prior = np.random.rand(6, 5)
prior_perfect = np.array([[1, 0, 0, 0, 0],
                             [0, 0, 1, 0, 0],
                             [0, 0, 1, 0, 0],
                             [1, 0, 0, 0, 0],
                             [0, 0, 1, 0, 0],
                             [0, 0, 1, 0, 0]])
# Emprical Priors
part_map = part_data['prior'] # Participant's maximum a priori
temp = 3/2 # Must be explored further
empirical_priors, entropy = discrete_empirical_priors(part_map, links, temp)
## Final prior assignment
prior = empirical_priors
# Prior sample size
## MUST BE 1 WHEN USING EMPIRICAL PRIORS
prior_sample_size = 1
#print(prior**prior_sample_size)
# Ground truth model
## Import from behavioural experiment
gt_behavioural_exp = part_data['ground_truth']
## Any model as np.ndarray
custom_model = np.array([-1, 0, .0, -1, 0, 0])
## Final ground truth assignment
true_model = gt_behavioural_exp
# Action parameters
## Number of model to sample from the posterior
C = 1
## Different action possibility
poss_actions = np.arange(-100, 100)
poss_actions = np.array([85, 45, 0, -45, -85])
poss_actions = np.arange(-100, 101, step=10)
## Define action length (TO BE REFINED FOR FITTING DATA)
action_len = 5 
## Policy for action selection from action values
policy_funcs = softmax_policy_init(1) # Returns three function: sample, pmf, params
## Parameters
epsilon = 1e-2 # Certainty threshold: agent stops intervening after entropy goes below epsilon
knowledge = False  # Can be a model as nd.array, True for perfect knowledge, 'random' for random sampling and False for posterior based sampling
## Special parameters for tree searches
horizon = 1e-1 # For soft horizon discounted gain
discount = 0.1 # For soft horizon discounted gain
depth = 0 # Horizon for hard horizon undiscounted gain
## General behaviour parameter
behaviour = 'actor'   # Can be 'obs', 'random' or 'actor'


dict_keys(['trial_index', 'data', 'inters', 'inters_fit', 'standard_order', 'in_trial_order', 'prior', 'posterior', 'ground_truth'])


In [11]:
sensory_state = Omniscient_ST(N, K)
action_state = Discounted_gain_soft_horizon_TSAS(N, K, behaviour, poss_actions, action_len, policy_funcs, epsilon, C, knowledge, discount, horizon)
internal_state = Normative_DIS(N, K, prior, prior_sample_size, links, dt, theta, sigma, sample_params=True)

agent = Agent(N, sensory_state, internal_state, action_state)

if fitting:
    external_state = OU_Network(N, K, true_model, theta, dt, sigma, data=data, inters=inters, inters_fit=inters_fit)
else:
    external_state = OU_Network(N, K, true_model, theta, dt, sigma)

experiment = Experiment(agent, external_state)


In [12]:
# Run experiment
if fitting:
    experiment.fit(posterior)
else:
    experiment.run()

#experiment.entropy_report()

Compute action values, C= 0 Model n: 0 Sampled graph: [-1.   1.   1.   1.   0.5  1. ]


IndexError: index 64 is out of bounds for axis 0 with size 64

In [19]:
LL = internal_state._posterior_params_history[150]
LL_n = LL - np.amax(LL)
lh = np.exp(LL_n) / np.exp(LL_n).sum()

print(lh)
print(LL_n)

internal_state._models_to_links(lh)

[0. 0. 0. ... 0. 0. 0.]
[-48245.87949599 -21604.54075854 -20257.3368764  ... -24768.98664172
 -18417.89534877 -41502.72154174]


array([[1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        1.00000000e+000, 4.38976664e-204],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        1.00000000e+000, 4.38976664e-204],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 1.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        1.00000000e+000, 4.38976664e-204]])